# Echo-A 1B5 (Memory Finetune 3)
Fine tune 3 process, scaling up the context size of finetune 2

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [1]:
# # Init required dirs
# !mkdir -p ../../../model/
# !mkdir -p ../../../datapath/
# !mkdir -p ../../../checkpoint/

# # Download the Stage2.pth file
# !rm -rf ../../../model/Echo-A-1B5-Tune2.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Tune2.pth
# !ls -alh ../../../model/Echo-A-1B5-Tune2.pth

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [11]:
%%script bash
# Reset the dataset dir
mkdir -p ./dataset
rm -rf ./dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# And a mix of a larger word dataset (to help with generalization)
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-2-pc-count.jsonl   2  20000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-5-pc-count.jsonl   5  20000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-10-pc-count.jsonl  10 30000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-15-pc-count.jsonl  15 30000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-20-pc-count.jsonl  20 30000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-40-pc-count.jsonl  40 30000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-80-pc-count.jsonl  80 20000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-100-pc-count.jsonl 100 10000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-200-pc-count.jsonl 200 10000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-300-pc-count.jsonl 300 5000 &
python ./memory_script/gen_full_prompt_completion_jsonl.py ./dataset/full-word-400-pc-count.jsonl 400 5000 &

# Shuffled word list, helps gurantee all eligible tokens are seen
# as the RNG nature of gen, makes it possible for tokens to be missed
#
# This may intrdocue a sequencing bias if overused, but avoid the model from getting
# blind-sided by a token it has never seen before in the word list
python ./memory_script/shuffle_full_prompt_completion_jsonl.py ./dataset/shuf-word-25-pc-count.jsonl 25 5 &
python ./memory_script/shuffle_full_prompt_completion_jsonl.py ./dataset/shuf-word-50-pc-count.jsonl 50 5 &
python ./memory_script/shuffle_full_prompt_completion_jsonl.py ./dataset/shuf-word-100-pc-count.jsonl 100 5 &

wait
echo "## Done ##"

ls -alh ./dataset/

## Generating word reptition dataset ##


Generated JSONL file with - 2 max words, 20000 samples - at ./dataset/full-word-2-pc-count.jsonl
Generated JSONL file with - 5 max words, 20000 samples - at ./dataset/full-word-5-pc-count.jsonl
Generated JSONL file with - 15 max words, 30000 samples - at ./dataset/full-word-15-pc-count.jsonl
Generated JSONL file with - 100 max words, 10000 samples - at ./dataset/full-word-100-pc-count.jsonl
Generated JSONL file with - 20 max words, 30000 samples - at ./dataset/full-word-20-pc-count.jsonl
Generated JSONL file with - 10 max words, 30000 samples - at ./dataset/full-word-10-pc-count.jsonl
Generated JSONL file with - 80 max words, 20000 samples - at ./dataset/full-word-80-pc-count.jsonl
Generated JSONL file with - 40 max words, 30000 samples - at ./dataset/full-word-40-pc-count.jsonl
Generated JSONL file with - 300 max words, 5000 samples - at ./dataset/full-word-300-pc-count.jsonl
Generated JSONL file with - 400 max words, 5000 samples - at ./dataset/full-word-400-pc-count.jsonl
Generated 

## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [2]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/picocreator-memory-experiment/notebook/experiment/memory-enwiki
TRAINER_DIR: /root/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /root/picocreator-memory-experiment


## Stage 3 : Simple Memory finetuning

In [4]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-3.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-finetune-3/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 122.59it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-02e3a4888f99dd6d/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 102.52it/s]


In [3]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-3.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-3 (bs=256, train-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=1024

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 744904170
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 744904170
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230713_042913-orijy0ra
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run (8x3090) Echo-A-1B5 - Mem-Finetune-3 (bs=256, train-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/orijy0ra
Using /root/.cache/torch_extensions/py311

In [4]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/Echo-A-1B5-mem-finetune-3/last.ckpt" \
        "../model/Echo-A-1B5-Tune3.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/Echo-A-1B5-Tune3.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/Echo-A-1B5-mem-finetune-3/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/Echo-A-1B5-Tune3.pth
-rw-r--r-- 1 root root 5.7G Jul 13 06:38 ../model/Echo-A-1B5-Tune3.pth


In [5]:
# Lets do a quick dragon prompt validation
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B5-Tune3.pth"

Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module wkv_cuda...
RWKV_JIT_ON 1 RWKV_CUDA_ON 1 RESCALE_LAYER 0

Loading /root/picocreator-memory-experiment/model/Echo-A-1B5-Tune3.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float32, float32], store 25 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-fl